<a href="https://colab.research.google.com/github/dixit2003/Transfer-Learning-Approach-for-Melanoma-Skin-Cancer-Detection/blob/main/Model_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Melanomam Skin Cancer Detection** - Model


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
resnet = ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=IMAGE_SIZE+[3],
    pooling=None,
    classes=2
    )

resnet.output

In [ ]:
for layer in resnet.layers:
  layer.trainable=False

In [ ]:
resnet_model = Sequential()

In [ ]:
# Add a Flatten layer
resnet_model.add(resnet)
resnet_model.add(GlobalAveragePooling2D())
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

In [ ]:
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Melanoma Detection Folder/Preprocessed Dataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Melanoma Detection Folder/Preprocessed Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle=False)


In [ ]:
val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/Melanoma Detection Folder/Preprocessed Dataset/Val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle=False)


In [ ]:
history = resnet_model.fit(
  train_set,
  validation_data=val_set,
  epochs=50)

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

Accuracy


In [ ]:
test_loss, test_accuracy = resnet_model.evaluate(test_set)
print(test_accuracy)
print(test_loss)

In [ ]:
test_steps_per_epoch = np.math.ceil(test_set.samples / test_set.batch_size)
predictions = resnet_model.predict(test_set, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys())

cm = confusion_matrix(true_classes, predicted_classes)

fig, ax = plt.subplots(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
resnet_model.save_weights("model_weights.h5")

In [ ]:
# Create the model architecture
model = resnet_model

# Load the saved weights
model.load_weights("model_weights.h5")

In [ ]:
# image=cv2.imread('/content/drive/MyDrive/Melanoma Detection Folder/Preprocessed Dataset/Val/benign/15.jpg')
# image_resized= cv2.resize(image, (224, 224))
# image=np.expand_dims(image_resized,axis=0)
# print(image.shape)

In [ ]:
# pred=resnet_model.predict(image)
# print(pred)

In [ ]:
# output_class=np.argmax(pred)
# print("The predicted class is", output_class)